# Connect to Label Studio

In [1]:
from label_studio_sdk.client import LabelStudio

ls = LabelStudio(
    base_url='http://localhost:8080',
    api_key='27c982caa9e599c9d81b25b00663e7d4f82c9e3c',
)

# Create New Project

First we need to define a **Labeling Interface** for the new project:

In [3]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import choices

label_config = LabelInterface.create({
    'text': 'Text',
    'label': choices(['Positive', 'Negative'])
})
print(label_config)

<View>
  <Text name="text" value="$text"/>
  <Choices name="label" toName="text">
    <Choice value="Positive"/>
    <Choice value="Negative"/>
  </Choices>
</View>


You can manually create [XML configuration for the labeling interface](https://labelstud.io/tags), but `LabelInterface` class provides a convenient way to create it programmatically.

Now let's create a new project with the specified labeling interface:

In [ ]:
project = ls.projects.create(
    title='Text Classification',
    label_config=label_config
)

print(f'Created project:\n{ls._client_wrapper.get_base_url()}/projects/{project.id}')

# Import Tasks to the Project

In [ ]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world"},
        {"text": "Hello Label Studio"},
        {"text": "What a beautiful day"},
    ]
)

## Import Tasks with Pre-annotations

In [ ]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world", "sentiment": "Positive"},
        {"text": "Goodbye Label Studio", "sentiment": "Negative"},
        {"text": "What a beautiful day", "sentiment": "Positive"},
    ],
    preannotated_from_fields=['sentiment']
)

More customizable way to import preannotations:

In [ ]:
from label_studio_sdk.label_interface.objects import PredictionValue

# this returns the same `LabelInterface` object as above
li = ls.projects.get(id=project.id).get_label_interface()

# by specifying what fields to `include` we can speed up task loading
for task in ls.tasks.list(project=project.id, include=["id"]):
    task_id = task.id
    prediction = PredictionValue(
        # tag predictions with specific model version string
        # it can help managing multiple models in Label Studio UI
        model_version='my_model_v1',
        # define your labels here
        result=[
            li.get_control('label').label(['Positive']),
        ]
    )
    ls.predictions.create(task=task_id, **prediction.model_dump())


# Export Annotations

In [ ]:
for task in ls.tasks.list(project=project.id):
    print(task.id)
    print(task.annotations)
    print(task.predictions)

Read more about task, annotations and predictions format in Label Studio [documentation](https://labelstud.io/guide/export#Label-Studio-JSON-format-of-annotated-tasks).